<a href="https://colab.research.google.com/github/RITabayuni/Skripsi_Perbandingan_K-Prototypes_Agglomerative-Gower/blob/main/PreprocessingData_Rahimi_Illong_Tabayuni_225150707111046.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***LIBRARY PREPARATION***

In [1]:
from google.colab import drive, files
import glob, os, pandas as pd
import numpy as np
import math

**NORMALISASI KOLOM**

In [2]:

drive.mount('/content/drive')
FOLDER = "/content/drive/MyDrive/"

paths = sorted(set(
    glob.glob(os.path.join(FOLDER, "*.csv")) +
    glob.glob(os.path.join(FOLDER, "*.CSV")) +
    glob.glob(os.path.join(FOLDER, "*.xlsx")) +
    glob.glob(os.path.join(FOLDER, "*.xls"))
))
print(f"Found {len(paths)} files")

DROP_POLICY = "either"        # baris dihapus jika system atau location kosong
TREAT_UNKNOWN_AS_MISSING = False  # "Unknown" tidak dianggap missing

# Kolom yang akan dihapus
DROP_COLS = [
    "accept_date","domain","ip_address","web_server","reason",
    "hackmode","mirror_image","state","def_grade","defacement_id"
]

BLANK_TOKENS = {"", "nan", "none", "null", "-", "--", "n/a", "na", "?"}

def read_any(p):
    if p.lower().endswith((".xlsx", ".xls")):
        return pd.read_excel(p)
    for enc in ("utf-8","ISO-8859-1","utf-16"):
        try:
            return pd.read_csv(p, encoding=enc)
        except Exception:
            continue
    return pd.read_csv(p, engine="python", encoding_errors="ignore")

def norm_colnames(df): #normalisasi kolom
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df



Mounted at /content/drive
Found 34 files


**DROP MISSING VALUES**

In [3]:
def is_missing_series(s: pd.Series, treat_unknown=False):
    if s is None:
        return pd.Series(True, index=range(0))
    # to string lower
    s_str = s.astype(str).str.strip().str.lower()
    miss = s.isna() | s_str.isin(BLANK_TOKENS)
    if treat_unknown:
        miss = miss | (s_str == "unknown")
    return miss

monthly_frames = []
summary = []

for p in paths:
    print("Reading", p)
    df = read_any(p)
    if df is None or len(df) == 0:
        print("  [skip empty]")
        continue

    df = norm_colnames(df)

    for col in ["system", "location"]:
        if col not in df.columns:
            df[col] = pd.NA

    n0 = len(df)

    # drop kolom yang tidak dipakai
    to_drop = [c for c in DROP_COLS if c in df.columns]
    if to_drop:
        df = df.drop(columns=to_drop, errors="ignore")

    base = os.path.basename(p)
    df["ym"] = os.path.splitext(base)[0]
    df["source_file"] = base

    miss_system   = is_missing_series(df["system"],   treat_unknown=TREAT_UNKNOWN_AS_MISSING)
    miss_location = is_missing_series(df["location"], treat_unknown=TREAT_UNKNOWN_AS_MISSING)

    if DROP_POLICY == "both":
        to_drop_mask = miss_system & miss_location
    elif DROP_POLICY == "either":
        to_drop_mask = miss_system | miss_location
    else:
        raise ValueError("DROP_POLICY harus 'both' atau 'either'")

    dropped = int(to_drop_mask.sum())
    kept = n0 - dropped

    df = df.loc[~to_drop_mask].copy()

    summary.append({
        "file": base,
        "rows_before": n0,
        "dropped": dropped,
        "kept": kept,
        "drop_policy": DROP_POLICY,
        "treat_unknown_as_missing": TREAT_UNKNOWN_AS_MISSING
    })

    monthly_frames.append(df)

raw = pd.concat(monthly_frames, ignore_index=True) if monthly_frames else pd.DataFrame()
print("Total rows loaded after drop:", len(raw))


Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202301.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202302.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202303.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202304.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202305.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202306.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202307.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202308.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202309.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202310.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/Defacements202311.csv
Reading /content/drive/MyDrive/KULIAH/Skripsi/Defacements/Full/De

**DEDUPLIKASI DATA**

In [4]:
COL_YM        = "ym"
COL_ATTACKER  = "attacker"
COL_LOCATION  = "location"
COL_TYPE      = "type"
COL_ADD_DATE  = "add_date"
COL_REDEF     = "redefacement"
COL_SYSTEM    = "system"


# Deduplikasi:
# - Mass akan dedup pakai level tanggal (YYYY-MM-DD) secara default
# - Regular akan dedup pakai timestamp penuh secara default
DEDUP_USE_ADD_DATE_DAY_MASS     = True
DEDUP_USE_ADD_DATE_DAY_REGULAR  = False


def ensure_datetime_day_key(s: pd.Series) -> pd.Series:
    if np.issubdtype(s.dtype, np.datetime64):
        return s.dt.date.astype(str)
    return pd.to_datetime(s, errors="coerce").dt.date.astype(str)


def to_string_key(s: pd.Series) -> pd.Series:
    if np.issubdtype(s.dtype, np.datetime64):
        return s.astype("datetime64[ns]").astype(str)
    return s.astype(str)


def is_mass_type(series: pd.Series) -> pd.Series:
    return series.astype(str).str.lower().str.contains("mass")


def dedup_all_global(df: pd.DataFrame) -> pd.DataFrame:
    required = [COL_TYPE, COL_ATTACKER, COL_ADD_DATE, COL_LOCATION, COL_REDEF, COL_SYSTEM]
    missing = [c for c in required if c not in df.columns]
    if missing:
        print(f"[WARN] Lewatkan dedup karena kolom wajib hilang: {missing}")
        return df

    # Total Sebelum
    before_total = len(df)

    mask_mass = is_mass_type(df[COL_TYPE])

    # key untuk MASS
    if DEDUP_USE_ADD_DATE_DAY_MASS:
        add_key_mass = ensure_datetime_day_key(df[COL_ADD_DATE])
    else:
        add_key_mass = to_string_key(df[COL_ADD_DATE])

    # key untuk REGULAR
    if DEDUP_USE_ADD_DATE_DAY_REGULAR:
        add_key_reg = ensure_datetime_day_key(df[COL_ADD_DATE])
    else:
        add_key_reg = to_string_key(df[COL_ADD_DATE])

    # MASS
    mass_part = df[mask_mass].copy()
    if not mass_part.empty:
        mass_part = mass_part.assign(_add_key=add_key_mass[mask_mass])
        mass_part = mass_part.sort_values(
            [COL_ATTACKER, "_add_key", COL_LOCATION, COL_REDEF, COL_SYSTEM],
            kind="stable"
        )
        before_mass = len(mass_part)
        mass_part = mass_part.drop_duplicates(
            subset=[COL_ATTACKER, "_add_key", COL_LOCATION, COL_REDEF, COL_SYSTEM],
            keep="first"
        ).drop(columns=["_add_key"], errors="ignore")
        after_mass = len(mass_part)
    else:
        before_mass = after_mass = 0

    # REGULAR
    regular_part = df[~mask_mass].copy()
    if not regular_part.empty:
        regular_part = regular_part.assign(_add_key=add_key_reg[~mask_mass])
        regular_part = regular_part.sort_values(
            [COL_TYPE, COL_ATTACKER, "_add_key", COL_LOCATION, COL_REDEF, COL_SYSTEM],
            kind="stable"
        )
        before_reg = len(regular_part)
        regular_part = regular_part.drop_duplicates(
            subset=[COL_TYPE, COL_ATTACKER, "_add_key", COL_LOCATION, COL_REDEF, COL_SYSTEM],
            keep="first"
        ).drop(columns=["_add_key"], errors="ignore")
        after_reg = len(regular_part)
    else:
        before_reg = after_reg = 0

    out = pd.concat([regular_part, mass_part], ignore_index=True)

    # Total Sesudah
    after_total = len(out)
    removed_total = before_total - after_total

    print(f"[DEDUP] Total: {before_total:,} -> {after_total:,} (removed {removed_total:,})")
    print(f"[DEDUP] Regular: {before_reg:,} -> {after_reg:,} (removed {before_reg-after_reg:,})")
    print(f"[DEDUP] Mass: {before_mass:,} -> {after_mass:,} (removed {before_mass-after_mass:,})")

    return out

raw = dedup_all_global(raw)



[DEDUP] Total: 331,901 -> 141,560 (removed 190,341)
[DEDUP] Regular: 144,668 -> 131,182 (removed 13,486)
[DEDUP] Mass: 187,233 -> 10,378 (removed 176,855)


In [5]:
raw.to_csv("data_after_dedup.csv", index=False)
print("Saved combined full data -> data_after_dedup.csv")

Saved combined full data -> data_after_dedup.csv


**SAMPLING**

In [6]:
import numpy as np
import pandas as pd

COL_YM = "ym"
RANDOM_STATE = 42
FRAC_TOTAL = 0.10

def extract_year_month(df, col_ym="ym"):
    s = df[col_ym].astype(str)
    yyyymm = s.str.extract(r"(\d{6})$", expand=False)
    df["year"]  = pd.to_numeric(yyyymm.str[:4], errors="coerce")
    df["month"] = pd.to_numeric(yyyymm.str[4:6], errors="coerce")
    return df

# Total sampel
N = len(raw)
n_total = int(round(FRAC_TOTAL * N))

# Ukuran strata tiap bulan
counts = raw.groupby(COL_YM).size().sort_index()

#Proportional allocation
alloc_float = counts / counts.sum() * n_total
alloc = np.floor(alloc_float).astype(int)

remainder = n_total - alloc.sum()
if remainder > 0:
    frac = (alloc_float - np.floor(alloc_float)).sort_values(ascending=False)
    for ym in frac.index[:remainder]:
        alloc.loc[ym] += 1

# Random sampling tiap bulan
parts = []
for ym, g in raw.groupby(COL_YM):
    nh = int(min(alloc.loc[ym], len(g)))
    if nh > 0:
        parts.append(g.sample(n=nh, replace=False, random_state=RANDOM_STATE))

sampled = pd.concat(parts, ignore_index=True)

# Buat kolom year & month
sampled = extract_year_month(sampled, col_ym=COL_YM)

print("Total data:", N)
print("Total sampel:", len(sampled))
print("Persentase:", 100 * len(sampled) / N)
print(sampled.groupby(COL_YM).size().sort_index())

# Drop kolom metadata yang tidak dipakai
DROP_AFTER = ["ym", "source_file"]
sampled_cluster = sampled.drop(columns=[c for c in DROP_AFTER if c in sampled.columns], errors="ignore")

# Simpan
sampled_cluster.to_csv("zoneh_stratified_10%.csv", index=False)
print("Saved -> zoneh_stratified_10%.csv")


Total data: 141560
Total sampel: 14156
Persentase: 10.0
ym
Defacements202301    841
Defacements202302    579
Defacements202303    802
Defacements202304    676
Defacements202305    569
Defacements202306    390
Defacements202307    483
Defacements202308    514
Defacements202309    366
Defacements202310    328
Defacements202311    353
Defacements202312    230
Defacements202401    217
Defacements202402    385
Defacements202403    498
Defacements202404    433
Defacements202405    424
Defacements202406    371
Defacements202407    302
Defacements202408    345
Defacements202409    299
Defacements202410    261
Defacements202411    163
Defacements202412    147
Defacements202501    129
Defacements202502    257
Defacements202503    245
Defacements202504    420
Defacements202505    426
Defacements202506    424
Defacements202507    440
Defacements202508    358
Defacements202509    515
Defacements202510    966
dtype: int64
Saved -> zoneh_stratified_10%.csv


**CEK COVERAGE (OPSIONAL)**

In [7]:
from scipy.stats import chi2_contingency

COL_YM = "ym"
COL_TYPE = "type"
COL_SYSTEM = "system"
COL_REDEF = "redefacement"
COL_ATTACKER = "attacker"
COL_LOCATION = "location"

TVD_GOOD = 0.05
TVD_OK   = 0.10

def _norm_series(s: pd.Series) -> pd.Series:
    return s.fillna("NA").astype(str)

def distribution(df: pd.DataFrame, col: str) -> pd.Series:
    s = _norm_series(df[col])
    return s.value_counts(normalize=True)

def tvd(p: pd.Series, q: pd.Series) -> float:
    all_idx = p.index.union(q.index)
    p2 = p.reindex(all_idx).fillna(0.0)
    q2 = q.reindex(all_idx).fillna(0.0)
    return 0.5 * np.abs(p2 - q2).sum()

def compare_distributions(pop: pd.DataFrame, samp: pd.DataFrame, col: str, topn_report: int = 15) -> dict:
    p = distribution(pop, col)
    q = distribution(samp, col)

    all_idx = p.index.union(q.index)
    dfc = pd.DataFrame({
        "pop_pct": (p.reindex(all_idx).fillna(0.0) * 100),
        "samp_pct": (q.reindex(all_idx).fillna(0.0) * 100),
    })
    dfc["diff_pp"] = (dfc["pop_pct"] - dfc["samp_pct"]).abs()

    tvd_val = tvd(p, q)
    max_diff = float(dfc["diff_pp"].max())
    worst = dfc.sort_values("diff_pp", ascending=False).head(topn_report)

    if tvd_val < TVD_GOOD:
        status = "sangat mirip"
    elif tvd_val < TVD_OK:
        status = "cukup mirip"
    else:
        status = "ada perbedaan"

    return {"col": col, "tvd": tvd_val, "max_diff_pp": max_diff, "status": status,
            "detail": dfc.sort_values("diff_pp", ascending=False),
            "worst_top": worst}

def coverage_by_month(pop: pd.DataFrame, samp: pd.DataFrame, ym_col: str = COL_YM) -> pd.DataFrame:
    pop_n = pop.groupby(ym_col).size().rename("pop_n")
    samp_n = samp.groupby(ym_col).size().rename("samp_n")
    out = pd.concat([pop_n, samp_n], axis=1).fillna(0).astype(int)
    out["sampling_rate"] = np.where(out["pop_n"] > 0, out["samp_n"] / out["pop_n"], np.nan)
    return out.sort_index()

def chi_square_test(pop: pd.DataFrame, samp: pd.DataFrame, col: str, min_count: int = 5):
    pop_s = _norm_series(pop[col])
    samp_s = _norm_series(samp[col])

    vc = pd.concat([pop_s, samp_s]).value_counts()
    keep = vc[vc >= min_count].index

    pop_f = pop_s.where(pop_s.isin(keep), "OTHER_RARE")
    samp_f = samp_s.where(samp_s.isin(keep), "OTHER_RARE")

    tab = pd.crosstab(
        pd.Series(["pop"] * len(pop_f) + ["samp"] * len(samp_f), name="group"),
        pd.concat([pop_f, samp_f], ignore_index=True)
    )

    chi2, pval, dof, _ = chi2_contingency(tab.values)

    n = tab.values.sum()
    k = tab.shape[1]
    r = tab.shape[0]
    denom = n * (min(r - 1, k - 1))
    cramer_v = np.sqrt(chi2 / denom) if denom > 0 else np.nan
    return {"chi2": chi2, "pval": pval, "dof": dof, "cramers_v": cramer_v, "table": tab}

def high_cardinality_coverage(pop: pd.DataFrame, samp: pd.DataFrame, col: str) -> pd.DataFrame:
    """Ringkasan untuk kolom kategori besar (attacker/location) TANPA topk."""
    pop_s = _norm_series(pop[col])
    samp_s = _norm_series(samp[col])

    pop_set = set(pop_s.unique())
    samp_set = set(samp_s.unique())

    inter = pop_set & samp_set
    union = pop_set | samp_set

    out = pd.DataFrame([{
        "col": col,
        "pop_unique": len(pop_set),
        "samp_unique": len(samp_set),
        "coverage_pop_categories_in_sample": (len(inter) / len(pop_set)) if len(pop_set) else np.nan,
        "jaccard_overlap": (len(inter) / len(union)) if len(union) else np.nan,
    }])
    return out

def run_checks_no_topk(pop: pd.DataFrame, samp: pd.DataFrame):
    print("\n=== (1) Coverage per bulan (YM) ===")
    cov = coverage_by_month(pop, samp, ym_col=COL_YM)
    display(cov)

    print("\n=== (2) TVD untuk kolom utama (type/system/redefacement) ===")
    cols = [c for c in [COL_TYPE, COL_SYSTEM, COL_REDEF] if c in pop.columns and c in samp.columns]
    summaries = []
    for col in cols:
        res = compare_distributions(pop, samp, col, topn_report=15)
        summaries.append({"Kolom": col, "TVD": round(res["tvd"], 4),
                          "Max selisih (pp)": round(res["max_diff_pp"], 2),
                          "Status": res["status"]})
        print(f"\n[{col}] TVD={res['tvd']:.4f} | Max Δpp={res['max_diff_pp']:.2f} | {res['status']}")
        display(res["worst_top"])

    display(pd.DataFrame(summaries).sort_values(["TVD", "Max selisih (pp)"], ascending=False))

    print("\n=== (3) Kolom high-cardinality (attacker/location) - ringkasan tanpa topk ===")
    for col in [COL_ATTACKER, COL_LOCATION]:
        if col in pop.columns and col in samp.columns:
            display(high_cardinality_coverage(pop, samp, col))

    print("\n=== (4) Chi-square (opsional; lihat Cramer's V) ===")
    for col in cols:
        test = chi_square_test(pop, samp, col, min_count=5)
        print(f"[{col}] p-value={test['pval']:.3e}, Cramer's V={test['cramers_v']:.4f}")

    return {"coverage": cov}

# Jalankan:
out_checks = run_checks_no_topk(raw, sampled)



=== (1) Coverage per bulan (YM) ===


,pop_n,samp_n,sampling_rate
ym,,,
Defacements202301,8410,841,0.100000
Defacements202302,5793,579,0.099948
Defacements202303,8023,802,0.099963
Defacements202304,6761,676,0.099985
Defacements202305,5689,569,0.100018
Defacements202306,3904,390,0.099898
Defacements202307,4826,483,0.100083
Defacements202308,5140,514,0.100000
Defacements202309,3660,366,0.100000



=== (2) TVD untuk kolom utama (type/system/redefacement) ===

[type] TVD=0.0000 | Max Δpp=0.00 | sangat mirip


,pop_pct,samp_pct,diff_pp
type,,,
regular,92.668833,92.66742,0.001413
mass,7.331167,7.33258,0.001413



[system] TVD=0.0089 | Max Δpp=0.78 | sangat mirip


...

,Kolom,TVD,Max selisih (pp),Status
1,system,0.0089,0.78,sangat mirip
2,redefacement,0.0016,0.16,sangat mirip
0,type,0.0000,0.00,sangat mirip



=== (3) Kolom high-cardinality (attacker/location) - ringkasan tanpa topk ===


,col,pop_unique,samp_unique,coverage_pop_categories_in_sample,jaccard_overlap
0,attacker,5521,1658,0.300308,0.300308


,col,pop_unique,samp_unique,coverage_pop_categories_in_sample,jaccard_overlap
0,location,182,139,0.763736,0.763736



=== (4) Chi-square (opsional; lihat Cramer's V) ===
[type] p-value=1.000e+00, Cramer's V=0.0000
[system] p-value=5.168e-01, Cramer's V=0.0098
[redefacement] p-value=6.330e-01, Cramer's V=0.0012
